In [1]:
import pynq
import numpy as np
import time, struct, os, mmap
import utils

In [2]:
bitfile = '../bitfiles/bram_intf/bram_intf.bit'
reg_base = 0x41c00000
mem_base = 0x41100000

length=4096

In [3]:
pynq.Overlay(bitfile)

In [6]:
#my own implementation
reg = utils.mmio(reg_base,16)
bram = utils.mmio(mem_base, length)

In [13]:
test_data = np.arange(64).astype('i4')-32
print('write data from axil bram')
bram.write(0, test_data, dtype='i4')
print('read data from axil bram')
out = bram.read(0, len(test_data)*4)
for i in range(len(out)):
    assert (test_data[i]==out[i])
print('pass\n')

print('Read data from axil reg')
reg.write(2*4,np.array(0))
for i in range(len(test_data)):
    reg.write(4, np.array(i))
    out = reg.read(4*3, 4)
    assert (out==test_data[i])
print('pass\n')

print('Write data from axil reg')
reg.write(2*4,np.array(1))
reg.write(4, np.array(0))
test_data = np.arange(64).astype('i4')
for i in range(len(test_data)):
    reg.write(4, np.array(i))
    reg.write(0, np.array(i))

print('Read from axil reg')
reg.write(2*4,np.array(0))
for i in range(len(test_data)):
    reg.write(4, np.array(i))
    out = reg.read(4*3, 4)
    assert (out==test_data[i])
print('pass\n')

print('read data from axil bram')
out = bram.read(0, len(test_data)*4)
for i in range(len(out)):
    assert (test_data[i]==out[i])
print('pass\n')
    

write data from axil bram
read data from axil bram
pass

Read data from axil reg
pass

Write data from axil reg
Read from axil reg
[0]
[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
[10]
[11]
[12]
[13]
[14]
[15]
[16]
[17]
[18]
[19]
[20]
[21]
[22]
[23]
[24]
[25]
[26]
[27]
[28]
[29]
[30]
[31]
[32]
[33]
[34]
[35]
[36]
[37]
[38]
[39]
[40]
[41]
[42]
[43]
[44]
[45]
[46]
[47]
[48]
[49]
[50]
[51]
[52]
[53]
[54]
[55]
[56]
[57]
[58]
[59]
[60]
[61]
[62]
[63]
pass

read data from axil bram
pass



In [33]:
##pynq functions
reg = pynq.MMIO(reg_base,8)
bram = pynq.MMIO(mem_base, length)
##some basic test
test_data = [x for x in range(64)]
bin_data = struct.pack('64I', *test_data)
print('Writing using axil bram')
bram.write(0, bin_data)

time.sleep(0.5)
print('Read using axil bram')
out = bram.array[:64]     #this always read it as 32-int :(
for i in range(len(out)):
    assert (i==out[i])



Writing using axil bram
Read using axil bram


ValueError: must have exactly one of create/read/write/append mode

In [10]:
#it seems thats better to map it myself >:(
mmap_file = os.open('/dev/mem', os.O_RDWR | os.O_SYNC)
mem = mmap.mmap(mmap_file, length,
                        mmap.MAP_SHARED,
                        mmap.PROT_READ | mmap.PROT_WRITE,
                        offset=mem_base)
os.close(mmap_file)
struct.unpack('64I',mem[:64*4])


(0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0)